In [2]:
#pandas已经安装好，可以直接导入
import pandas as pd
#使用get_price函数获得某只股票的数据
#start_date和end_date分别为起始日期与结束日期
#frequency参数是获取数据的周期，daily是日线数据
df = get_price('601318.XSHG',
              start_date = '2020-01-01',
              end_date = '2020-04-01',
              frequency = 'daily')
#检查一下载入情况
df.head()

,open,close,high,low,volume,money
2020-01-02,85.90,86.12,86.79,85.88,77825207.0,6.712532e+09
2020-01-03,86.81,86.20,86.88,85.90,59498001.0,5.137312e+09
2020-01-06,85.92,85.60,86.87,85.50,63644804.0,5.487968e+09
2020-01-07,86.01,86.15,86.46,85.67,45218832.0,3.886450e+09
2020-01-08,85.98,85.00,85.98,84.50,62805311.0,5.347387e+09


获取股票概况

In [3]:
#使用get_security_info函数可以获取股票概况
info = get_security_info('601318.XSHG')
#返回的对象包含若干属性
#包括股票名称display_name
print('股票的中文名称：',info.display_name)
#股票简称name
print('股票简称：',info.name)
#股票上市日期start_date
print('股票上市日期：',info.start_date)
#股票退市日期end_date，如未退市则显示2200-01-01
print('股票退市日期：',info.end_date)
#产品类型type，stock代表股票、etf代表ETF基金、index代表指数等
print('产品类型：',info.type)
#如果是分级基金，parent可查看母基金
print('产品的母基金：',info.parent)

股票的中文名称： 中国平安
股票简称： ZGPA
股票上市日期： 2007-03-01
股票退市日期： 2200-01-01
产品类型： stock
产品的母基金： None


In [4]:
#使用get_all_securities可获得全部证券信息
#支持使用切片的方式获得其中部分证券的信息
info_all = get_all_securities()[:5]
#查看获取的信息
info_all

,display_name,name,start_date,end_date,type
000001.XSHE,平安银行,PAYH,1991-04-03,2200-01-01,stock
000002.XSHE,万科A,WKA,1991-01-29,2200-01-01,stock
000004.XSHE,国农科技,GNKJ,1990-12-01,2200-01-01,stock
000005.XSHE,世纪星源,SJXY,1990-12-10,2200-01-01,stock
000006.XSHE,深振业A,SZYA,1992-04-27,2200-01-01,stock


In [5]:
info_index = get_all_securities(types=['etf'])[:5]
info_index

,display_name,name,start_date,end_date,type
159001.XSHE,保证金,BZJ,2014-10-20,2200-01-01,etf
159003.XSHE,招商快线,ZSKX,2014-10-20,2200-01-01,etf
159005.XSHE,添富快钱,TFKQ,2015-01-13,2200-01-01,etf
159801.XSHE,芯片基金,XPJJ,2020-02-18,2200-01-01,etf
159804.XSHE,创精选88,CJX88,2020-03-31,2200-01-01,etf


获取财务数据

In [6]:
#首先要创建一个query object对象
#获取平台valuation表中，代码为601318的股票数据
q = query(valuation).filter(valuation.code=='601318.XSHG')
#把query object对象传入到get_fundamentals函数
#并制定日期为2020年4月1日
df = get_fundamentals(q, '2020-04-01')
#查看返回的结果
df

,id,code,pubDate,pe_ratio,turnover_ratio,pb_ratio,ps_ratio,pcf_ratio,capitalization,market_cap,circulating_cap,circulating_market_cap,day,pe_ratio_lyr
0,53742808,601318.XSHG,2020-04-01,8.4814,0.5141,1.8824,1.0841,-278.0137,1828024.125,12671.8633,1083266.5,7509.2031,2020-04-01,8.4814


In [7]:
#创建一个query object
#制定获取的数据为股票代码
q = query(valuation.code,
          #动态市盈率
          valuation.pe_ratio,
          #市现率
          valuation.pcf_ratio,
          #和换手率
         valuation.turnover_ratio)\
.filter(
    #筛选条件为市盈率小于20且大于0
    valuation.pe_ratio < 20,
        valuation.pe_ratio > 0,
        #市现率大于0且小于20
       valuation.pcf_ratio > 0,
       valuation.pcf_ratio < 20,
        #换手率大于4%
       valuation.turnover_ratio > 4)\
.order_by(
    #按照换手率降序排列
    valuation.turnover_ratio.desc())
#使用get_fundamentals函数获得数据
portfolio = get_fundamentals(q, date = '2020-04-03')
#检查结果
portfolio.head(30)

,code,pe_ratio,pcf_ratio,turnover_ratio
0,002458.XSHE,8.5327,15.0275,9.7450
1,600387.XSHG,7.7496,4.9772,7.2717
2,300107.XSHE,10.5846,11.8711,5.3762
3,002234.XSHE,5.2662,17.8631,5.1201
4,002839.XSHE,10.9746,14.5165,4.9225
5,000610.XSHE,18.4441,11.9632,4.6329
6,603733.XSHG,19.6847,7.6782,4.3345
7,002869.XSHE,8.8296,6.0086,4.2709


In [8]:
#从jqdata导入finance包
from jqdata import finance

In [20]:
#创建一个query object,
#查询STK_SHAREHOLDER_TOP10表中的code字段
q = query(finance.STK_SHAREHOLDER_TOP10.code,
          #以及shareholder_rank字段
         finance.STK_SHAREHOLDER_TOP10.shareholder_rank,
          #shareholder_name字段
         finance.STK_SHAREHOLDER_TOP10.shareholder_name,
          #shareholder_class字段
         finance.STK_SHAREHOLDER_TOP10.shareholder_class,
          #还有share_ration字段
         finance.STK_SHAREHOLDER_TOP10.share_ratio)\
#过滤条件是股票代码002458
.filter(finance.STK_SHAREHOLDER_TOP10.code == '002458.XSHE',
        #发布市间晚于2020年1月1日
       finance.STK_SHAREHOLDER_TOP10.pub_date > '2020-01-01')
#执行这个query，返回一个dataframe
shareholders = finance.run_query(q)
#查看返回的结果
shareholders

,code,shareholder_rank,shareholder_name,shareholder_class,share_ratio
0,002458.XSHE,1,曹积生,自然人,41.64
1,002458.XSHE,2,迟汉东,自然人,3.09
2,002458.XSHE,3,香港中央结算有限公司,其他机构,1.89
3,002458.XSHE,4,李玲,自然人,1.41
4,002458.XSHE,5,耿培梁,自然人,1.16
5,002458.XSHE,6,中国农业银行股份有限公司-银华内需精选混合型证券投资基金（LOF）,证券投资基金,1.09
6,002458.XSHE,7,东方邦信创业投资有限公司,风险投资,1.02
7,002458.XSHE,8,柳炳兰,自然人,0.96
8,002458.XSHE,9,交通银行股份有限公司-工银瑞信双利债券型证券投资基金,证券投资基金,0.88
9,002458.XSHE,10,李秀国,自然人,0.79


In [40]:
#创建一个query object
#查询STK_SHAREHOLDERS_SHARE_CHANGE表的code字段
q = query(finance.STK_SHAREHOLDERS_SHARE_CHANGE.code,
          #pub_date字段
          finance.STK_SHAREHOLDERS_SHARE_CHANGE.pub_date,
          #shareholder_name字段
         finance.STK_SHAREHOLDERS_SHARE_CHANGE.shareholder_name,
          #type字段
         finance.STK_SHAREHOLDERS_SHARE_CHANGE.type,
          #change_number字段
         finance.STK_SHAREHOLDERS_SHARE_CHANGE.change_number,
          #change_ratio字段
         finance.STK_SHAREHOLDERS_SHARE_CHANGE.change_ratio,
          #after_change_ratio字段
         finance.STK_SHAREHOLDERS_SHARE_CHANGE.after_change_ratio)\
#筛选条件是股票代码002458
.filter(finance.STK_SHAREHOLDERS_SHARE_CHANGE.code == '002458.XSHE',
        #且发布日期晚于2019年9月1日
       finance.STK_SHAREHOLDERS_SHARE_CHANGE.pub_date > '2019-09-01')
#执行这个query，返回一个dataframe
shrchg = finance.run_query(q)
#查看返回的结果
shrchg

,code,pub_date,shareholder_name,type,change_number,change_ratio,after_change_ratio
0,002458.XSHE,2019-09-25,耿培梁,1,122000.0,0.021,NaN
1,002458.XSHE,2019-09-25,耿培梁,1,719300.0,0.125,NaN
2,002458.XSHE,2019-09-25,耿培梁,1,10000.0,0.002,NaN
3,002458.XSHE,2019-09-25,纪永梅,1,2100.0,0.000,NaN
4,002458.XSHE,2019-09-25,耿培梁,1,62000.0,0.011,NaN
5,002458.XSHE,2019-09-25,耿培梁,1,234000.0,0.041,NaN
6,002458.XSHE,2019-09-25,耿培梁,0,782600.0,0.136,1.168
7,002458.XSHE,2019-09-25,纪永梅,1,187770.0,0.033,NaN
8,002458.XSHE,2019-09-25,耿培梁,1,782600.0,0.136,NaN
9,002458.XSHE,2020-02-22,赵桂苹,0,1000.0,0.000,NaN


In [45]:
#从jgdata中导入全部函数
from jqdata import *

In [50]:
#使用get_money_flow函数获取002458的资金流向数据
mf = get_money_flow('002458.XSHE',
                   #截止日期为2020年4月3日
                   end_date = '2020-04-03',
                    #获取字段包括日期
                   fields = ['date',
                             #股票代码
                           'sec_code',
                             #涨跌幅
                           'change_pct',
                             #主力金额，包括超大单和大单
                           'net_amount_main',
                             #主力成交额占总成交额的比例
                           'net_pct_main'],
                    #获取10个交易日的数据
                   count = 10)
#查看结果
mf

,date,sec_code,change_pct,net_amount_main,net_pct_main
0,2020-03-23,002458.XSHE,-9.99,-11561.8543,-17.0788
1,2020-03-24,002458.XSHE,0.53,-4068.5285,-7.1877
2,2020-03-25,002458.XSHE,7.25,1817.5482,2.1306
3,2020-03-26,002458.XSHE,2.36,-2352.3292,-2.5584
4,2020-03-27,002458.XSHE,-0.85,-5059.1633,-8.0721
5,2020-03-30,002458.XSHE,6.96,4476.8310,4.2767
6,2020-03-31,002458.XSHE,8.57,-2550.2537,-1.6243
7,2020-04-01,002458.XSHE,1.00,-2896.5712,-1.7715
8,2020-04-02,002458.XSHE,-1.09,-10572.1955,-9.4150
9,2020-04-03,002458.XSHE,-0.37,-4050.7753,-3.4890
